In [2]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
from time import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['GORDON_REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import scipy.ndimage as nd
import scipy

from skimage.feature import hog
from skimage import data, color, exposure
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.metrics import confusion_matrix

import random
import pickle

In [32]:
class Classifier:
    classes = ['12N', '5N', '7n', '7N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 'Bg'];
    def __init__(self, training_X, clf, vocabulary):
        self.training_X = training_X;
        self.clf = clf;
        self.vocabulary = vocabulary;
    
    def generateKernel(self, test_X):
        n_testing = test_X.shape[0];
        n_training = self.training_X.shape[0];
        testKernel = np.zeros(shape=(n_testing, n_training));
        for i in range(0, n_testing):
            for j in range(0, n_training):
                testKernel[i][j] = sum(np.minimum(test_X[i], self.training_X[j]));
        return testKernel;
    
    def getHistogram(self, image, kmeans, M, L):
        x_size = image.shape[1]; # Column
        y_size = image.shape[0]; # Row
        # Get SIFT key points and descriptor.
        sift = cv2.SIFT();
        key_points, descriptor = sift.detectAndCompute(image, None);
        # Map key points with type.
        if descriptor is None:
            return None;
        cluster_labels = kmeans.predict(descriptor);
        cluster_labels = cluster_labels;
        # Generate position list of key_points
        positions = np.asarray([kp.pt for kp in key_points], np.int)
        # Level 0
        weight_0 = 1.0 / (2**L);
        histogram = np.bincount(cluster_labels, minlength = M);
        histogram = histogram * weight_0;
        # Other levels
        for l in range(1, L):
            weight = 1.0 / (2**(L - l + 1));
            grid_size_x = x_size / (2**l);
            grid_size_y = y_size / (2**l);
            grid_boundaries_x = range(0, x_size, grid_size_x);
            grid_boundaries_y = range(0, y_size, grid_size_y);
            number_of_grid = (2**l)**2;
            grid_crs = positions / [grid_size_x, grid_size_y];
            grid_cols = grid_crs[:,0];  # x
            grid_rows = grid_crs[:,1];  # y
    
            keypoint_grid_indices = grid_rows * (2**l) + grid_cols;
            hists = np.zeros(shape=(number_of_grid, M), dtype=np.int);
            for ki, label in zip(keypoint_grid_indices, cluster_labels):
                hists[ki][label] += 1;
            hists = hists * weight;
            histogram = np.hstack((histogram, hists.flatten()));
        return (histogram / len(key_points)); # Normalization

    def predictLabel(self, image):
        M = self.vocabulary.shape[0];
        L = 3;
        kmeans = KMeans(init='random', n_clusters=M, n_init=10);
        kmeans.fit(self.vocabulary);
        img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY);
        histogram = self.getHistogram(img, kmeans, M, L);
        testKernel = self.generateKernel(np.asarray(histogram).reshape(1, histogram.shape[0]));
        return Classifier.classes[self.clf.predict(testKernel)[0]];

In [24]:
patches = np.load('/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/patches/patches_Pn_138.npy');

In [33]:
myClassifier = pickle.load(open('/oasis/projects/nsf/csd395/ruogu/svm/classifier.p', "rb"));

In [34]:
print myClassifier.predictLabel(patches[10]);

Pn
